In [1]:
#import
import pandas as pd
import numpy as np
import torch
import os
import random
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
from tqdm import tqdm

# for graphing
import seaborn as sns
import matplotlib.pyplot as plt

/home/kelly790/.conda/envs/torch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#ID는 필요없어서 제거
df = pd.read_csv('/home/kelly790/dacon 연습/문장분류/train.csv')
df.drop(columns=['ID'], inplace=True)

test = pd.read_csv('/home/kelly790/dacon 연습/문장분류/test.csv')
test.drop(columns=['ID'], inplace=True)

submission = pd.read_csv('/home/kelly790/dacon 연습/문장분류/sample_submission.csv')

In [30]:
#baseline과 같은 기본 seed 고정 및 device cuda 설정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

CFG = {
    'EPOCHS':10,
    'LEARNING_RATE':1e-5,
    'BATCH_SIZE':16,
    'SEED':41
}

seed_everything(CFG['SEED']) # Seed 고정
device = torch.device('cuda:1')
device

device(type='cuda', index=1)

In [31]:
#데이터 split
train,val,_,_ = train_test_split(df, df['label'], test_size=0.2, random_state=CFG['SEED'])

#train, valid 인덱스 섞여있으므로 reset
train = train.reset_index(drop=True)
valid = val.reset_index(drop=True)

huggingface 모델 검색 \
-> https://huggingface.co/models

In [32]:
#klue/roberta-small 사용

model = 'klue/roberta-small'
base_model = AutoModel.from_pretrained(model)
tokenizer = AutoTokenizer.from_pretrained(model)

Some weights of the model checkpoint at klue/roberta-small were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

In [33]:
class SentenceTypeDataset(Dataset):
    def __init__(self, dataframe, tokenizer, labels=None): #x,y가 뭔지 정의할 것임 ; dataframe, tokenizer, label은 parameter로
        texts = dataframe['문장'].values.tolist() #문장을 모두 리스트화

        self.texts = [tokenizer(text, padding='max_length', max_length=90, truncation=True, return_tensors='pt') for text in texts] 
        #텍스트마다 토큰화, 문장 길이가 다를 수 있어서 90으로 제한, 그보다 작은 건 padding으로 채우기, truncation으로 긴 문장 자르기, return_tensors는 tensor로 tokenizer 값 리턴
        
        self.labels = labels #라벨은 그대로

    def __len__(self):
        return len(self.texts) #text의 길이 리턴

    def __getitem__(self, idx): #idx에 해당되는 x,y를 리턴
        text = self.texts[idx] #x는 self.text에서 가져옴

        if self.labels is not None: #y는 텍스트(x)에서 해당되는 type, polarity, tense, certainty 가져옴
            type_tmp = self.labels['type'][idx]
            polarity_tmp = self.labels['polarity'][idx]
            tense_tmp = self.labels['tense'][idx]
            certainty_tmp = self.labels['certainty'][idx]
            return text, torch.Tensor(type_tmp), torch.Tensor(polarity_tmp), torch.Tensor(tense_tmp), torch.Tensor(certainty_tmp) 
            #dictionary에 저장된 형태는 list이고 pytorch에는 tensor 형태로 넣어야하므로 torch.Tensor()을 사용
        else:
            return text, torch.Tensor([-1,-1,-1,-1]), torch.Tensor([-1,-1,-1]), torch.Tensor([-1,-1,-1]), torch.Tensor([-1,-1])
            #test set의 경우 labels가 주어지지 않기 때문에 똑같은 길이지만 -1로 채운 tensor를 리턴해준다

In [34]:
class SentenceClassifier(nn.Module): 
    def __init__(self, base_model): 
        super().__init__() #nn.Module의 child class이므로 super().__init__()을 부르고
        self.klue = base_model # from transformers package 
        #base_model을 통해서 받을 pretrained_model을 self.klue에 저장

        #transfer learning의 기본적 방법은 중간에 hidden layer는 pretrained model의 것을 이용하고 output layer를 내가 원하는 방향으로 만들어서 training하는 것이다.
        self.fc1 = nn.Linear(768, 32) 
        self.relu = nn.ReLU() #activation function
        #multilabel classification이므로 각 label마다 (32,~) 으로 레이어를 만들어줌
        #~만큼의 out_feature가 필요한 이유는 types들을 one-hot encoding해줬기 때문
        self.type_clf = nn.Linear(32,4)
        self.polarity_clf = nn.Linear(32,3)
        self.tense_clf = nn.Linear(32,3)
        self.certainty_clf = nn.Linear(32,2)
        self.softmax = nn.Softmax(dim=1) #classification에서 많이 사용

    def forward(self, input_ids, attention_mask):
        # input_ids : token's id / attention_mask : make a model to focus on which token
        klue_out = self.klue(input_ids= input_ids, attention_mask = attention_mask)[0][:,0]
                            #토큰에 해당되는 ids, 어느 토큰에 집중해야하는지 : padding에서 추가된 padding token에 대한 접근을 막기 위해

        x = self.fc1(klue_out) 
        x = self.relu(x)

        type_output = self.type_clf(x)
        type_output = self.softmax(type_output)
        polarity_output = self.polarity_clf(x)
        polarity_output = self.softmax(polarity_output)
        tense_output = self.tense_clf(x)
        tense_output = self.softmax(tense_output)
        certainty_output = self.certainty_clf(x)
        certainty_output = self.softmax(certainty_output)

        return type_output, polarity_output, tense_output, certainty_output #multilabel이므로 4개의 값을 리턴, 단순한 binary 또는 multiclassification이면 보통 1개 output만 리턴

In [56]:
#training
def sentence_train(model, train_dataloader, val_dataloader, learning_rate, epochs, model_nm):
    best_val_loss = 99999999999999 # setting max (act as infinity)
    early_stopping_threshold_count = 0

    criterion = {
        'type' : nn.CrossEntropyLoss().to(device),
        'polarity' : nn.CrossEntropyLoss().to(device),
        'tense' : nn.CrossEntropyLoss().to(device),
        'certainty' : nn.CrossEntropyLoss().to(device)
    } #loss function임. 4개 다른 라벨이라서 dictionary에 4개 넣어줌. crossentropy를 통해 true값과 pred 값의 차이를 구하고 어떤 방향으로 weight를 조정해야하는지 정한다.

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    model = model.to(device)

    for epoch in range(epochs):
        total_acc_train = 0
        total_loss_train = 0 #epoch별로 loss값이 어땠는지 기록하기 위해 사용
        
        model.train() # sets into the training mode 
        #val이나 test set을 모델에 넘겨줄 때 model.eval() 불러야 weights들이 업데이트 되지 않는다.
        
        for train_input, type_label, polarity_label, tense_label, certainty_label in tqdm(train_dataloader): #train_dataloader는 지정해준 batch_size만큼 item의 x와 y를 넘겨준다. 여기서 SentenceTypeDataset의 getitem()이 쓰임 
            attention_mask = train_input['attention_mask'].to(device)
            input_ids = train_input['input_ids'].squeeze(1).to(device)
            type_label = type_label.to(device)
            polarity_label = polarity_label.to(device)
            tense_label = tense_label.to(device)
            certainty_label = certainty_label.to(device)

            optimizer.zero_grad() #매 epoch마다 0베이스에서 시작하게 함. 정확한 업뎃을 위해 꼭 필요
            
            type_output, polarity_output, tense_output, certainty_output = model(input_ids, attention_mask) # from the forward function
            #model에 input_ids와 attention_mask를 넣어 얻은 4 값을 저장한다. 이 값들은 각 라벨마다 원핫인코딩된 컬럼에 해당될 확률들
            
            loss = 0.25*criterion['type'](type_output, type_label.float()) + \
                   0.25*criterion['polarity'](polarity_output, polarity_label.float()) + \
                   0.25*criterion['tense'](tense_output, tense_label.float()) + \
                   0.25*criterion['certainty'](certainty_output, certainty_label.float())
            total_loss_train += loss.item() #criterion에 있는 crossentropyloss로 들어가 실제와 얼마나 유사한지 계산되고 그 계산된 값을 total_loss_train에 저장

            loss.backward()
            optimizer.step()
            #backpropagation 과정을 통해 weights들을 업데이트한다.
            

        with torch.no_grad(): # since we should not change gradient for validation 
            total_acc_val = 0
            total_loss_val = 0
            
            model.eval() # deactivate training
            #with 과정을 통해 weights들을 업데이트가 아닌 선언을 한다.
            
            # same process as the above
            for val_input, vtype_label, vpolarity_label, vtense_label, vcertainty_label in tqdm(val_dataloader):
                attention_mask = val_input['attention_mask'].to(device)
                input_ids = val_input['input_ids'].squeeze(1).to(device)

                vtype_label = vtype_label.to(device)
                vpolarity_label = vpolarity_label.to(device)
                vtense_label = vtense_label.to(device)
                vcertainty_label = vcertainty_label.to(device)
                
                vtype_output, vpolarity_output, vtense_output, vcertainty_output = model(input_ids, attention_mask) # from the forward function

                loss = 0.25*criterion['type'](vtype_output, vtype_label.float()) + \
                        0.25*criterion['polarity'](vpolarity_output, vpolarity_label.float()) + \
                        0.25*criterion['tense'](vtense_output, vtense_label.float()) + \
                        0.25*criterion['certainty'](vcertainty_output, vcertainty_label.float())

                total_loss_val += loss.item()
                
                #train과 같은 과정을 거치는데 여기서는 optimizer.step()이 없다. weight 업데이트를 하지 않기 때문

            
            print(f'Epochs: {epoch + 1} '
                  f'| Train Loss: {total_loss_train / len(train_dataloader): .3f} '
                  f'| Train Accuracy: {total_acc_train / (len(train_dataloader.dataset)): .3f} '
                  f'| Val Loss: {total_loss_val / len(val_dataloader): .3f} '
                  f'| Val Accuracy: {total_acc_val / len(val_dataloader.dataset): .3f}')

            if best_val_loss > total_loss_val:
                best_val_loss = total_loss_val # saving only the best one
                torch.save(model, f"/home/kelly790/dacon 연습/문장분류/{model_nm}.pt")
                print("Saved model")
                early_stopping_threshold_count = 0
                   
            else:
                early_stopping_threshold_count += 1 # checking how many epochs have passed that val_loss didn't increase
                
            if early_stopping_threshold_count >= 3: # ==> patience=1
                print("Early stopping")
                break
            #loss와 metric을 print하고 val_loss가 좋아졌는지 여부에 따라 모델을 저장할지 early stop할 것인지 정함


In [57]:
train_tmp = train[['문장', '유형', '극성', '시제', '확실성']] #label을 제외한 나머지 컬럼들만 킵하고 
train_tmp = pd.get_dummies(train_tmp, columns=['유형', '극성', '시제', '확실성']) #pd.get_dummies로 원핫인코딩
train_tmp

,문장,유형_대화형,유형_사실형,유형_예측형,유형_추론형,극성_긍정,극성_미정,극성_부정,시제_과거,시제_미래,시제_현재,확실성_불확실,확실성_확실
0,용산구청 관계자는 ＂재정이 열악한 지자체로서는 1800억원을 마련할 수 없다＂며 서...,0,1,0,0,1,0,0,1,0,0,0,1
1,부산시는 이처럼 부산이 가파른 상승세를 보이는 이유에 대해 지난해부터 추진하고 있는...,0,1,0,0,1,0,0,1,0,0,0,1
2,"그러나 미숙아, 만성호흡기질환, 선천 심장병, 선천 면역결핍질환, 암환자 등의 고위...",0,1,0,0,1,0,0,0,0,1,0,1
3,탁구 종목에서 중국 대표팀 위상이 뛰어나기 때문이다.,0,0,0,1,1,0,0,0,0,1,0,1
4,이 논문에 따르면 ＇BT-11＇은 뇌의 신경전달물질인 아세틸콜린을 분해하는 효소의 ...,0,1,0,0,1,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13227,우리가 익히 아는 대로 임꺽정은 신출귀몰했다.,0,1,0,0,1,0,0,1,0,0,0,1
13228,김 상무보는 ＂실제 이용자 수와 인당 사용시간 등 주요 데이터가 매년 두 자릿수 상...,0,1,0,0,1,0,0,1,0,0,0,1
13229,＇디폴트 옵션＇의 필요성을 주장해온 쪽이 항상 사례로 들어온 것이 ＇401K＇로 불...,1,0,0,0,1,0,0,0,0,1,0,1
13230,1992년부터 선양시 조선족노인협회를 후원하기 시작해 1997년에는 1500㎡ 건물...,0,1,0,0,1,0,0,1,0,0,0,1


In [58]:
#각 라벨별 뽑아서 train_labels에 dictionary 형태로 저장
train_type = train_tmp.iloc[:,1:5].values.tolist()
train_polarity = train_tmp.iloc[:,5:8].values.tolist()
train_tense = train_tmp.iloc[:,8:11].values.tolist()
train_certainty = train_tmp.iloc[:,11:13].values.tolist()
train_labels = {
    'type': train_type,
    'polarity': train_polarity,
    'tense': train_tense,
    'certainty': train_certainty
}

In [59]:
val_tmp = val[['문장', '유형', '극성', '시제', '확실성']]
val_tmp = pd.get_dummies(val_tmp, columns=['유형', '극성', '시제', '확실성'])

val_type = val_tmp.iloc[:,1:5].values.tolist()
val_polarity = val_tmp.iloc[:,5:8].values.tolist()
val_tense = val_tmp.iloc[:,8:11].values.tolist()
val_certainty = val_tmp.iloc[:,11:13].values.tolist()
val_labels = {
    'type': val_type,
    'polarity': val_polarity,
    'tense': val_tense,
    'certainty': val_certainty
}

In [60]:
train_dataloader = DataLoader(SentenceTypeDataset(train_tmp, tokenizer, train_labels), batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0) # num_workers: how many subprocesses to use for data loading  
val_dataloader = DataLoader(SentenceTypeDataset(val_tmp, tokenizer, val_labels), batch_size=CFG['BATCH_SIZE'], num_workers=0)

#데이터셋이 준비가 되었으므로 SentenceTypeDataset에 dataframe, tokenizer, labels들을 넣고 batch_size 정하고 dataloader에 넣는다.
#dataloader는 지정된 batch만큼 items를 모델에 넘겨주어 training 한다.

In [61]:
model = SentenceClassifier(base_model)

In [62]:
sentence_train(model, train_dataloader, val_dataloader, CFG['LEARNING_RATE'], CFG['EPOCHS'], 'kclue')

100%|██████████| 207/207 [00:02<00:00, 98.91it/s]


Epochs: 1 | Train Loss:  0.732 | Train Accuracy:  0.000 | Val Loss:  0.664 | Val Accuracy:  0.000
Saved model


100%|██████████| 207/207 [00:02<00:00, 89.55it/s]


Epochs: 2 | Train Loss:  0.638 | Train Accuracy:  0.000 | Val Loss:  0.636 | Val Accuracy:  0.000
Saved model


100%|██████████| 207/207 [00:02<00:00, 90.96it/s]


...

Epochs: 8 | Train Loss:  0.583 | Train Accuracy:  0.000 | Val Loss:  0.616 | Val Accuracy:  0.000
Saved model


100%|██████████| 207/207 [00:02<00:00, 90.93it/s]


Epochs: 9 | Train Loss:  0.581 | Train Accuracy:  0.000 | Val Loss:  0.617 | Val Accuracy:  0.000


100%|██████████| 207/207 [00:02<00:00, 91.32it/s]

Epochs: 10 | Train Loss:  0.578 | Train Accuracy:  0.000 | Val Loss:  0.617 | Val Accuracy:  0.000


In [69]:
def get_type_predictions(model, loader):

    device = torch.device('cuda:1')
    model = model.to(device)
    
    type_probs, polarity_probs, tense_probs, clarity_probs = [], [], [], []
    with torch.no_grad():
        model.eval()
        for data_input, _, _, _, _ in tqdm(loader):
            attention_mask = data_input['attention_mask'].to(device)
            input_ids = data_input['input_ids'].squeeze(1).to(device)


            type_output, polarity_output, tense_output, clarity_output = model(input_ids, attention_mask)
            type_probs.append(type_output)
            polarity_probs.append(polarity_output)
            tense_probs.append(tense_output)
            clarity_probs.append(clarity_output)
    
    return torch.cat(type_probs).cpu().detach().numpy(), \
            torch.cat(polarity_probs).cpu().detach().numpy(), \
            torch.cat(tense_probs).cpu().detach().numpy(), \
            torch.cat(clarity_probs).cpu().detach().numpy()

In [70]:
model = torch.load("/home/kelly790/dacon 연습/문장분류/kclue.pt")
test_dataloader = DataLoader(SentenceTypeDataset(test, tokenizer), batch_size=CFG['BATCH_SIZE'], shuffle=False)

In [71]:
test_pred_type, test_pred_polarity, test_pred_tense, test_pred_certainty = get_type_predictions(model, test_dataloader)

100%|██████████| 444/444 [00:04<00:00, 106.66it/s]


In [72]:
test_type = ['대화형' if i==0 else '사실형' if i==1 else '예측형' if i==2 else '추론형' for i in [np.argmax(p) for p in test_pred_type]]
test_polarity = ['긍정' if i==0 else '미정' if i==1 else '부정' for i in [np.argmax(p) for p in test_pred_polarity]]
test_tense = ['과거' if i==0 else '미래' if i==1 else '현재' for i in [np.argmax(p) for p in test_pred_tense]]
test_certainty = ['불확실' if i==0 else '확실' for i in [np.argmax(p) for p in test_pred_certainty]]

In [73]:
label_sum = []
for i in range(len(test_type)):
    label_sum.append(f'{test_type[i]}-{test_polarity[i]}-{test_tense[i]}-{test_certainty[i]}')

submission['label'] = label_sum
submission.to_csv('/home/kelly790/dacon 연습/문장분류/sample_submission.csv', index=False)

In [74]:
submission

,ID,label
0,TEST_0000,사실형-긍정-현재-확실
1,TEST_0001,사실형-긍정-현재-확실
2,TEST_0002,사실형-긍정-과거-확실
3,TEST_0003,사실형-긍정-과거-확실
4,TEST_0004,사실형-긍정-과거-확실
...,...,...
7085,TEST_7085,사실형-긍정-현재-확실
7086,TEST_7086,추론형-긍정-현재-확실
7087,TEST_7087,사실형-긍정-미래-확실
7088,TEST_7088,추론형-긍정-미래-확실
